In [78]:
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from prophet import Prophet
import holidays
from sklearn.metrics import mean_absolute_percentage_error
from datetime import datetime, timedelta

# Load and preprocess the data
file_path = './sales.csv'
data = pd.read_csv(file_path)
data['BUSINESS_DATE'] = pd.to_datetime(data['BUSINESS_DATE'])
unique_store_ids = data['STORE_KEY'].unique()

# Load store information
store_info = pd.read_csv('./store.csv')
store_info = store_info.set_index('STORE_KEY')['STORE_NAME'].to_dict()

# US holidays
us_holidays = holidays.US()

# Create a DataFrame of holidays for Prophet
holiday_df = pd.DataFrame({
    'ds': pd.to_datetime(list(us_holidays.keys())),
    'holiday': list(us_holidays.values())
})

# Function to calculate MAPE and MdAPE safely
def calculate_errors(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero = (y_true != 0)
    percentage_errors = np.abs((y_true[non_zero] - y_pred[non_zero]) / y_true[non_zero]) * 100
    mape = np.mean(percentage_errors)
    mdape = np.median(percentage_errors)
    return mape, mdape

# Initialize the Dash app
app = dash.Dash(__name__)

# Custom CSS for the dropdown
dropdown_style = {
    'backgroundColor': 'white',
    'color': 'black',
    'border': '1px solid #ddd',
    'borderRadius': '5px',
    'padding': '5px'
}

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Store Sales Dashboard", style={'color': 'white', 'textAlign': 'center'}),
    
    # Dropdown to select store
    html.Div([
        html.Label("Select Store:", style={'color': 'white', 'marginRight': '10px'}),
        dcc.Dropdown(
            id='store-dropdown',
            options=[{'label': store_info.get(store_id, f'Store {store_id}'), 'value': store_id} for store_id in unique_store_ids],
            value=unique_store_ids[0],
            style=dropdown_style,
            clearable=False
        )
    ], style={'marginBottom': '20px'}),
    
    # Graphs
    dcc.Graph(id='sales-trend-graph'),
    dcc.Graph(id='forecast-graph'),
    
    # Metrics
    html.Div(id='metrics-display')
], style={'backgroundColor': '#2B2B2B', 'color': 'white', 'minHeight': '100vh', 'padding': '20px'})

# Callback to update the sales trend graph
@app.callback(
    Output('sales-trend-graph', 'figure'),
    Input('store-dropdown', 'value')
)
def update_sales_trend(selected_store):
    store_data = data[data['STORE_KEY'] == selected_store]
    time_series_data = store_data.groupby('BUSINESS_DATE')['NET_SALES_FINAL_USD_AMOUNT'].sum().reset_index()
    
    store_name = store_info.get(selected_store, f'Store {selected_store}')
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time_series_data['BUSINESS_DATE'], y=time_series_data['NET_SALES_FINAL_USD_AMOUNT'],
                             mode='lines', name='Daily Sales'))
    fig.update_layout(
        title=f'Sales Trend for {store_name}',
        xaxis_title='Date',
        yaxis_title='Sales (USD)',
        paper_bgcolor='#2B2B2B',
        plot_bgcolor='#2B2B2B',
        font_color='white'
    )
    return fig

# Callback to update the forecast graph and metrics
@app.callback(
    [Output('forecast-graph', 'figure'),
     Output('metrics-display', 'children')],
    Input('store-dropdown', 'value')
)
def update_forecast_and_metrics(selected_store):
    store_data = data[data['STORE_KEY'] == selected_store]
    time_series_data = store_data.groupby('BUSINESS_DATE')['NET_SALES_FINAL_USD_AMOUNT'].sum().reset_index()
    
    store_name = store_info.get(selected_store, f'Store {selected_store}')
    
    # Prepare data for Prophet
    prophet_data = time_series_data.rename(columns={'BUSINESS_DATE': 'ds', 'NET_SALES_FINAL_USD_AMOUNT': 'y'})
    
    # Add holiday feature
    prophet_data['is_holiday'] = prophet_data['ds'].isin(holiday_df['ds']).astype(int)
    
    # Split the data into training and test sets
    train_size = int(len(prophet_data) * 0.8)
    train_data = prophet_data[:train_size]
    test_data = prophet_data[train_size:]
    
    # Initialize and fit Prophet model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        holidays=holiday_df
    )
    model.add_regressor('is_holiday')
    
    model.fit(train_data)
    
    # Make predictions
    future = model.make_future_dataframe(periods=90)
    future['is_holiday'] = future['ds'].isin(holiday_df['ds']).astype(int)
    forecast = model.predict(future)
    
    # Create forecast plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=prophet_data['ds'], y=prophet_data['y'], mode='lines', name='Actual'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill=None, mode='lines', line_color='rgba(0,100,80,0.2)', name='Lower Bound'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='lines', line_color='rgba(0,100,80,0.2)', name='Upper Bound'))
    fig.update_layout(
        title=f'Sales Forecast for {store_name}',
        xaxis_title='Date',
        yaxis_title='Sales (USD)',
        paper_bgcolor='#2B2B2B',
        plot_bgcolor='#2B2B2B',
        font_color='white'
    )
    
    # Calculate metrics
    test_forecast = forecast.tail(len(test_data))
    mape, mdape = calculate_errors(test_data['y'], test_forecast['yhat'])
    total_sales = store_data['NET_SALES_FINAL_USD_AMOUNT'].sum()
    
    metrics_html = html.Div([
        html.H3(f"Metrics for {store_name}"),
        html.P(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%"),
        html.P(f"Median Absolute Percentage Error (MdAPE): {mdape:.2f}%"),
        html.P(f"Total Sales: ${total_sales:.2f}")
    ])
    
    return fig, metrics_html
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

    

/home/satyam/Desktop/anaconda3/envs/trilytics/lib/python3.8/site-packages/prophet/forecaster.py:243: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

18:09:17 - cmdstanpy - INFO - Chain [1] start processing
18:09:17 - cmdstanpy - INFO - Chain [1] done processing
/home/satyam/Desktop/anaconda3/envs/trilytics/lib/python3.8/site-packages/prophet/forecaster.py:243: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

18:09:19 - cmdstanpy - INFO - Chain [1] start processing
18:09:19 - cmdstanpy - INFO - Chain [1] done processing
/home/satyam/Desktop/anaconda3/envs/trilytics/lib/python3.8/site-packages/prophet/forecaster.py:243: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

18:09:33 - cmdstanpy - INFO - Chain [1] start processing
18:09:33 - cmdstanpy - INFO - 

# Sales Forecasting for the Next 3 Months

In [77]:
# Load and preprocess data
file_path = './sales.csv'
data = pd.read_csv(file_path)
data['BUSINESS_DATE'] = pd.to_datetime(data['BUSINESS_DATE'])
time_series_data = data.groupby('BUSINESS_DATE')['NET_SALES_FINAL_USD_AMOUNT'].sum().reset_index()
time_series_data.columns = ['ds', 'y']

us_holidays = holidays.US()
time_series_data['is_holiday'] = time_series_data['ds'].apply(lambda x: 1 if x in us_holidays else 0)

# Train model
model = Prophet(holidays=pd.DataFrame({'ds': list(us_holidays.keys()), 'holiday': list(us_holidays.values())}))
model.add_regressor('is_holiday')
model.fit(time_series_data)

# Generate future forecast
future = model.make_future_dataframe(periods=90)
future['is_holiday'] = future['ds'].apply(lambda x: 1 if x in us_holidays else 0)
forecast = model.predict(future)

# Initialize Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1("Sales Forecast Dashboard", style={'color': 'white', 'textAlign': 'center'}),
    dcc.Graph(id='sales-trend-graph'),
    dcc.Graph(id='forecast-graph'),
    html.Div(id='metrics-display'),
    html.Img(id='components-plot'),
    html.H3("Forecast for the next 90 days:", style={'color': 'white'}),
    html.Div(id='forecast-table')
], style={'backgroundColor': '#2B2B2B', 'color': 'white', 'minHeight': '100vh', 'padding': '20px'})

# Callback for sales trend graph
@app.callback(Output('sales-trend-graph', 'figure'), Input('sales-trend-graph', 'id'))
def update_sales_trend(_):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time_series_data['ds'], y=time_series_data['y'], mode='lines', name='Daily Sales'))
    fig.update_layout(
        title='Historical Sales Trend',
        xaxis_title='Date',
        yaxis_title='Sales',
        paper_bgcolor='#2B2B2B',
        plot_bgcolor='#2B2B2B',
        font_color='white'
    )
    return fig

# Callback for forecast graph, metrics, components plot, and forecast table
@app.callback(
    [Output('forecast-graph', 'figure'),
     Output('metrics-display', 'children'),
     Output('components-plot', 'src'),
     Output('forecast-table', 'children')],
    Input('forecast-graph', 'id')
)
def update_forecast_and_metrics(_):
    # Forecast plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time_series_data['ds'], y=time_series_data['y'], mode='lines', name='Actual'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill=None, mode='lines', line_color='rgba(0,100,80,0.2)', name='Lower Bound'))
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='lines', line_color='rgba(0,100,80,0.2)', name='Upper Bound'))
    fig.update_layout(
        title='Sales Forecast',
        xaxis_title='Date',
        yaxis_title='Sales',
        paper_bgcolor='#2B2B2B',
        plot_bgcolor='#2B2B2B',
        font_color='white'
    )
    
    # Metrics display
    metrics_html = html.Div([
        html.H3("Forecast Metrics"),
        html.P(f"Train MAPE: {train_mape:.2f}%"),
        html.P(f"Train MdAPE: {train_mdape:.2f}%"),
        html.P(f"Train RMSE: ${train_rmse:.2f}"),
        html.P(f"Test MAPE: {test_mape:.2f}%"),
        html.P(f"Test MdAPE: {test_mdape:.2f}%"),
        html.P(f"Test RMSE: ${test_rmse:.2f}")
    ])
    
    # Components plot
    plt.figure(figsize=(12, 8))
    model.plot_components(forecast)
    plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format="png", dpi=300, bbox_inches='tight', facecolor='#2B2B2B', edgecolor='none')
    plt.close()
    data = base64.b64encode(buf.getbuffer()).decode("ascii")
    components_src = f"data:image/png;base64,{data}"
    
    # Forecast table
    forecast_table = html.Table(
        # Header
        [html.Tr([html.Th(col) for col in ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']])] +
        # Body
        [html.Tr([
            html.Td(forecast['ds'].iloc[i].strftime('%Y-%m-%d')),
            html.Td(f"${forecast['yhat'].iloc[i]:.2f}"),
            html.Td(f"${forecast['yhat_lower'].iloc[i]:.2f}"),
            html.Td(f"${forecast['yhat_upper'].iloc[i]:.2f}")
        ]) for i in range(len(forecast)-90, len(forecast))]
    )
    
    return fig, metrics_html, components_src, forecast_table

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


18:01:56 - cmdstanpy - INFO - Chain [1] start processing
18:01:57 - cmdstanpy - INFO - Chain [1] done processing


<Figure size 1200x800 with 0 Axes>

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[77], line 108, in update_forecast_and_metrics(_='forecast-graph')
     85 # # Components plot
     86 # plt.figure(figsize=(12, 8))
     87 # model.plot_components(forecast)
   (...)
     94 
     95 # Forecast table
     96 forecast_table = html.Table(
     97     # Header
     98     [html.Tr([html.Th(col) for col in ['Date', 'Forecast', 'Lower Bound', 'Upper Bound']])] +
   (...)
    105     ]) for i in range(len(forecast)-90, len(forecast))]
    106 )
--> 108 return fig, metrics_html, components_src, forecast_table
        fig = Figure({
    'data': [{'mode': 'lines',
              'name': 'Actual',
              'type': 'scatter',
              'x': array([datetime.datetime(2019, 4, 23, 0, 0),
                          datetime.datetime(2019, 4, 24, 0, 0),
                          datetime.datetime(2019, 4, 25, 0, 0), ...,